In [26]:
!pip install yfinance
!pip install bs4
!pip install nbformat
%pip install plotly
!pip install requests
!pip install beautifulsoup4 html5lib

In [5]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [6]:
import warnings
# ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [7]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

Question 1: Use yfinance to Extract Stock Data¶
Using the Ticker function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is TSLA.

In [10]:
# Question 1: Use yfinance to Extract Stock Data¶
# Using the Ticker function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is TSLA.
tsla_ticker = yf.Ticker("TSLA")
# get Tesla's historical stock data
tesla_data = tsla_ticker.history(period="1y")
print(tesla_data)

                                 Open        High         Low       Close  \
Date                                                                        
2023-11-08 00:00:00-05:00  223.149994  224.149994  217.639999  222.110001   
2023-11-09 00:00:00-05:00  219.750000  220.800003  206.679993  209.979996   
2023-11-10 00:00:00-05:00  210.029999  215.380005  205.690002  214.649994   
2023-11-13 00:00:00-05:00  215.600006  225.399994  211.610001  223.710007   
2023-11-14 00:00:00-05:00  235.029999  238.139999  230.720001  237.410004   
...                               ...         ...         ...         ...   
2024-11-01 00:00:00-04:00  252.039993  254.000000  246.630005  248.979996   
2024-11-04 00:00:00-05:00  244.559998  248.899994  238.880005  242.839996   
2024-11-05 00:00:00-05:00  247.339996  255.279999  246.210007  251.440002   
2024-11-06 00:00:00-05:00  284.670013  289.589996  275.619995  288.529999   
2024-11-07 00:00:00-05:00  288.890015  299.750000  285.519989  296.910004   

In [11]:
# Extract historical stock data for the maximum period and save it in a DataFrame
tesla_data = tsla_ticker.history(period="max")

# Display the first few rows of the data to confirm
print(tesla_data.head())

                               Open      High       Low     Close     Volume  \
Date                                                                           
2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667  281494500   
2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667  257806500   
2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000  123282000   
2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   77097000   
2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000  103003500   

                           Dividends  Stock Splits  
Date                                                
2010-06-29 00:00:00-04:00        0.0           0.0  
2010-06-30 00:00:00-04:00        0.0           0.0  
2010-07-01 00:00:00-04:00        0.0           0.0  
2010-07-02 00:00:00-04:00        0.0           0.0  
2010-07-06 00:00:00-04:00        0.0           0.0  


In [12]:
# Step 2: Extract historical stock data for the maximum period and save it in a DataFrame
tesla_data = tsla_ticker.history(period="max")

# Step 3: Reset the index of the DataFrame
tesla_data.reset_index(inplace=True)

# Step 4: Display the first five rows of the DataFrame
print(tesla_data.head())

                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  


Question 2: Use Webscraping to Extract Tesla Revenue Data¶
Use the requests library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm Save the text of the response as a variable named html_data.

In [14]:
# URL of the webpage to be downloaded
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"

# Send a GET request to the URL and store the response
response = requests.get(url)

# Save the text of the response as a variable named html_data
html_data = response.text

# Display a part of html_data to confirm
print(html_data[:500])  # Displaying the first 500 characters for a quick check



<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->
    <head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
		<link rel="canonical" href="https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue" />
	


In [15]:
# Parse the HTML data using BeautifulSoup
soup = BeautifulSoup(html_data, 'html.parser')

# Display the parsed HTML to confirm
print(soup.prettify()[:500])  # Displaying the first 500 characters for a quick check

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <link href="https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue" rel="canonical"/>
  <title>
   Te


In [18]:
# Step 0: Fetch the HTML data
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
response = requests.get(url)
html_data = response.text

# Step 1: Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_data, 'html.parser')

# Step 2: Create an empty DataFrame for Tesla revenue data
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

# Step 3: Locate the relevant table by its content
tables = soup.find_all('table')
revenue_table = None
for table in tables:
    if "Tesla Quarterly Revenue" in table.text:
        revenue_table = table
        break

# Step 4: Check if the Tesla Quarterly Revenue table was found
if revenue_table:
    # Step 5: Iterate through each row in the table's body
    rows = revenue_table.find('tbody').find_all('tr')
    for row in rows:
        # Step 6: Extract date and revenue data from each row
        cols = row.find_all('td')
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace('$', '').replace(',', '')  # Clean up the revenue data
        
        # Step 7: Append the data as a new row to the DataFrame using pd.concat()
        new_row = pd.DataFrame({"Date": [date], "Revenue": [revenue]})
        tesla_revenue = pd.concat([tesla_revenue, new_row], ignore_index=True)

# Step 8: Display the DataFrame
print(tesla_revenue)

          Date Revenue
0   2022-09-30   21454
1   2022-06-30   16934
2   2022-03-31   18756
3   2021-12-31   17719
4   2021-09-30   13757
5   2021-06-30   11958
6   2021-03-31   10389
7   2020-12-31   10744
8   2020-09-30    8771
9   2020-06-30    6036
10  2020-03-31    5985
11  2019-12-31    7384
12  2019-09-30    6303
13  2019-06-30    6350
14  2019-03-31    4541
15  2018-12-31    7226
16  2018-09-30    6824
17  2018-06-30    4002
18  2018-03-31    3409
19  2017-12-31    3288
20  2017-09-30    2985
21  2017-06-30    2790
22  2017-03-31    2696
23  2016-12-31    2285
24  2016-09-30    2298
25  2016-06-30    1270
26  2016-03-31    1147
27  2015-12-31    1214
28  2015-09-30     937
29  2015-06-30     955
30  2015-03-31     940
31  2014-12-31     957
32  2014-09-30     852
33  2014-06-30     769
34  2014-03-31     621
35  2013-12-31     615
36  2013-09-30     431
37  2013-06-30     405
38  2013-03-31     562
39  2012-12-31     306
40  2012-09-30      50
41  2012-06-30      27
42  2012-03

In [19]:
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(',', '').str.replace('$', '')

In [20]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [21]:
# 
# Display the last 5 row of the tesla_revenue dataframe using the tail function. Take a screenshot of the results.
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


In [22]:
# Create a ticker object for GameStop
gme_ticker = yf.Ticker("GME")

# Get GameStop's historical stock data
gamestop_data = gme_ticker.history(period="1y")
print(gamestop_data)

                                Open       High        Low      Close  \
Date                                                                    
2023-11-08 00:00:00-05:00  13.510000  13.760000  13.280000  13.280000   
2023-11-09 00:00:00-05:00  13.250000  13.320000  12.700000  12.700000   
2023-11-10 00:00:00-05:00  12.810000  12.970000  12.350000  12.540000   
2023-11-13 00:00:00-05:00  12.500000  12.530000  11.830000  12.140000   
2023-11-14 00:00:00-05:00  12.750000  13.390000  12.690000  12.900000   
...                              ...        ...        ...        ...   
2024-11-04 00:00:00-05:00  22.200001  22.650000  21.879999  22.330000   
2024-11-05 00:00:00-05:00  22.320000  23.200001  22.270000  22.990000   
2024-11-06 00:00:00-05:00  23.309999  23.820000  22.900000  23.100000   
2024-11-07 00:00:00-05:00  23.070000  23.660000  22.809999  23.450001   
2024-11-08 00:00:00-05:00  23.070000  23.450001  23.330000  23.424500   

                            Volume  Dividends  Sto

In [23]:
# Extract historical stock data for the maximum period and save it in a DataFrame
gme_data = gme_ticker.history(period="max")

# Display the first few rows of the data to confirm
gme_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2002-02-15 00:00:00-05:00,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [24]:
# Reset the index of the gme_data DataFrame
gme_data.reset_index(inplace=True)

# Display the first five rows of the gme_data DataFrame
print(gme_data.head())

                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620129  1.693350  1.603296  1.691667  76216000   
1 2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683250  11021600   
2 2002-02-15 00:00:00-05:00  1.683251  1.687459  1.658002  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615921  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  



Question 4: Use Webscraping to Extract GME Revenue Data¶
Use the requests library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html. Save the text of the response as a variable named html_data_2.

In [25]:
# URL of the webpage to be downloaded
url_2 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"

# Send a GET request to the URL and store the response
response_2 = requests.get(url_2)

# Save the text of the response as a variable named html_data_2
html_data_2 = response_2.text

# Display a part of html_data_2 to confirm
html_data_2[:500]  # Displaying the first 500 characters for a quick check

'<!DOCTYPE html>\n<!-- saved from url=(0105)https://web.archive.org/web/20200814131437/https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue -->\n<html class=" js flexbox canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface g'

In [27]:
# Assuming html_data contains the HTML content from the previous step
soup = BeautifulSoup(html_data, 'html.parser')  # or 'html5lib' as the parser

# Display a portion of the parsed HTML to confirm successful parsing
print(soup.prettify()[:500])  # Displaying the first 500 characters for a quick check

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <link href="https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue" rel="canonical"/>
  <title>
   Te


In [ ]:
# Step 0: Download the HTML data (use the appropriate URL for the GameStop data)
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
response = requests.get(url)
html_data = response.text

# Step 1: Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_data, 'html.parser')

# Step 2: Create an empty DataFrame for GameStop revenue data
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

# Step 3: Locate the GameStop Revenue table by checking for its text content
tables = soup.find_all('table')
revenue_table = None
for table in tables:
    if "GameStop Quarterly Revenue" in table.text:
        revenue_table = table
        break

# Step 4: Check if the GameStop Quarterly Revenue table was found
if revenue_table:
    # Step 5: Iterate through each row in the table's body
    rows = revenue_table.find('tbody').find_all('tr')
    for row in rows:
        # Step 6: Extract date and revenue data from each row
        cols = row.find_all('td')
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace('$', '').replace(',', '')  # Remove dollar sign and comma
        
        # Step 7: Append the data as a new row to the DataFrame using pd.concat()
        new_row = pd.DataFrame({"Date": [date], "Revenue": [revenue]})
        gme_revenue = pd.concat([gme_revenue, new_row], ignore_index=True)

# Step 8: Display the DataFrame
print(gme_revenue)

SyntaxError: invalid syntax (2523320663.py, line 36)